In [ ]:
pip install dlt[duckdb]

In [5]:
import math
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)
values = list(square_root_generator(limit))
s = math.fsum(values)
print(f"The sum is: {s}")

    

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
The sum is: 8.382332347441762


In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)
#ages = sum(Age)
#print("total ages are: {ages}")

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [21]:
import dlt
# define the connection to load to.
generator_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
# load  generator to a table at the pipeline destination:
info = generator_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

info = generator_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
print(info)
     

Pipeline dlt_ipykernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///C:\Users\HP\Documents\projects\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708083661.2317548 is LOADED and contains no failed jobs


In [26]:
import duckdb
conn = duckdb.connect(f"{generator_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generator_pipeline.dataset_name}'")
display(conn.sql("show tables"))

person = conn.sql("SELECT * FROM people")
display(person)

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

┌───────┬──────────┬───┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │ … │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │   │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │ … │ 1708083660.3192432 │ x6/o5npDz6qqcg │ NULL       │
│     2 │ Person_2 │ … │ 1708083660.3192432 │ 5Vt+JDOHlQv7Jg │ NULL       │
│     3 │ Person_3 │ … │ 1708083660.3192432 │ qkloC4L+kkd4tA │ NULL       │
│     4 │ Person_4 │ … │ 1708083660.3192432 │ 4GoFrJ9OxaTSfg │ NULL       │
│     5 │ Person_5 │ … │ 1708083660.3192432 │ VE/nXFvEY1MhwQ │ NULL       │
│     3 │ Person_3 │ … │ 1708083661.2317548 │ FJcP83aIOE8KPw │ Job_3      │
│     4 │ Person_4 │ … │ 1708083661.2317548 │ +gCXosp1i8/Tng │ Job_4      │
│     5 │ Person_5 │ … │ 1708083661.2317548 │ jjJrPD2V77fBXA │ Job_5      │
│     6 │ Person_6 │ … │ 1708083661.2317548 │ 7hsEHz0D9Rgczg │ Job_6      │
│     7 │ Pe

In [29]:
#new table to merge the two
info = generator_pipeline.run(people_1(),
										table_name="merge_people",
										write_disposition="replace",
                                        primary_key="id")
print(info)
info = generator_pipeline.run(people_2(),
										table_name="merge_people",
										write_disposition="merge",
                                        primary_key="id")
print(info)
#display all tables
display(conn.sql("show tables"))

Pipeline dlt_ipykernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///C:\Users\HP\Documents\projects\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708085135.5012312 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.55 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///C:\Users\HP\Documents\projects\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708085136.6143615 is LOADED and contains no failed jobs


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ merge_people        │
│ people              │
└─────────────────────┘

In [35]:
data = conn.sql("SELECT id, age From merge_people Order by ID ASC")
display(data)

┌───────┬───────┐
│  id   │  age  │
│ int64 │ int64 │
├───────┼───────┤
│     1 │    26 │
│     2 │    27 │
│     3 │    33 │
│     4 │    34 │
│     5 │    35 │
│     6 │    36 │
│     7 │    37 │
│     8 │    38 │
└───────┴───────┘

In [38]:
#total ages for unmerged table
ages1=conn.sql("SELECT sum(age) FROM people")
display(ages1)

#total ages for merged table
ages2=conn.sql("SELECT sum(age) FROM merge_people")
display(ages2)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘